In [114]:
!pip install langchain-community langchain-chroma langchain-openai bs4

In [115]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.callbacks import get_openai_callback
from dotenv import load_dotenv
import os

# [MY CODE] 환경변수 로드

In [116]:
load_dotenv()
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

# [MY CODE] RAG internet source 설정 및 로드

In [117]:
loader = WebBaseLoader(
    web_paths=("https://spartacodingclub.kr/blog/all-in-challenge_winner",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("css-18vt64m")
        )
    ),
)
docs = loader.load()
print(docs)

[Document(metadata={'source': 'https://spartacodingclub.kr/blog/all-in-challenge_winner'}, page_content="스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  493·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.🏆\xa0대상[Lexi Note] 언어공부 필기 웹 서비스서비스 제작자: 다나와(김다애, 박나경)💡W는 어문학을 전공하는 대학생입니다. 매일 새로운 단어와 문장 구조를 공부하고 있지만, 효율적으로 학습하는 것이 쉽지 않았습니다. 단어의 의미를 찾기 위해 사전을 뒤적이고, 긴 문장을 이해하려고 번역기를 사용하다 보면, 필기 노트는 어느새 뒷전으로 밀려났거든요. 사전, 번역기, 원서, 필기노트를 왔다 갔다 하다 보면 시간이 다 지나가 버리곤 했죠.W와 같이 어문 전공생은 문법, 어휘, 문장 구조 등 다양한 자료를 학습해야 합니다. 여러 자료를 번갈아 학습하다보니 ‘사전-번역기-원서-필기노트’ 왕복으로 학습 효율이 나지 않아 고민인 경우도 많으실 거예요. <Lexi Note>는 단어를 드래그하면 네이버 사전으로 바로 연동 돼 단어의 의미를 찾으며 동시에 필기 할 수 있어요. 이외에도 번역 버튼을 누르면 파파고 번역기가 연동돼 긴 문장도 쉽게 이해할 수 있어요. 언어 학습에 필요한 할일 목록과 스케줄 템플릿을 제공하여 효율적으로 공부할 수 

# [MY CODE] 스플리터

In [118]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200
)

splits = text_splitter.split_documents(docs)

# 문서 개수
print(len(splits))

# 전체 청크의 길이
total_chunk_size = sum(len(chunk.page_content) for chunk in splits)
print(f"전체 청크 사이즈: {total_chunk_size}")

3
전체 청크 사이즈: 5966


# [MY CODE] 임베딩 & 벡터 DB 저장

In [119]:
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(api_key=OPENAI_API_KEY)
)

# [MY CODE] LLM 모델

In [120]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)

# [MY CODE] 프롬프트 & 검색(Retrieval)

In [121]:
# 벡터스토어를 검색할 수 있는 리트리버 객체로 변환
retriever = vectorstore.as_retriever()

user_msg = "ALL-in 코딩 공모전 수상작들을 요약해줘."

# 리트리버를 사용하여 사용자 메시지와 관련된 문서 검색
retrieved_docs = retriever.invoke(user_msg)

def format_docs(docs):
    # 각 문서의 내용을 연결하여 하나의 텍스트로 반환
    return "\n\n".join(doc.page_content for doc in docs)

prompt = hub.pull("rlm/rag-prompt")

user_prompt = prompt.invoke({"context": format_docs(retrieved_docs), "question": user_msg})
print(user_prompt)

/opt/miniconda3/envs/hanghae99/lib/python3.10/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: ALL-in 코딩 공모전 수상작들을 요약해줘. \nContext: 스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  493·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.🏆\xa0대상[Lexi Note] 언어공부 필기 웹 서비스서비스 제작자: 다나와(김다애, 박나경)💡W는 어문학을 전공하는 대학생입니다. 매일 새로운 단어와 문장 구조를 공부하고 있지만, 효율적으로 학습하는 것이 쉽지 않았습니다. 단어의 의미를 찾기 위해 사전을 뒤적이고, 긴 문장을 이해하려고 번역기를 사용하다 보면, 필기 노트는 어느새 뒷전으로 밀려났거든요. 사전, 번역기, 원서, 필기노트를 왔다 갔다 하다 보면 시간이 다 지나가 버리곤 했죠.W와 같이 어문 전공생은 문법, 어휘, 문장 구조 등 다양한 자료를 학습해야 합니다. 여러 자료를 번갈아 학습하다보니 

# [MY CODE] 질문 & 응답 & 토큰 수 확인

In [122]:
# OpenAI API 호출과 토큰 추적
with get_openai_callback() as callback:
    response = llm.invoke(user_prompt)

    # 토큰 관련 정보 출력
    print(f"입력 토큰 수: {callback.prompt_tokens}")
    print(f"응답 토큰 수: {callback.completion_tokens}")
    print(f"총 토큰 수: {callback.total_tokens}")
    print(f"총 비용: ${callback.total_cost}")

입력 토큰 수: 10450
응답 토큰 수: 125
총 토큰 수: 10575
총 비용: $0.0009128999999999999


In [123]:
print(response.content)

ALL-in 코딩 공모전에서 수상작으로 선정된 프로젝트는 총 6개입니다. 대상 수상작은 언어 공부를 효율적으로 돕는 웹 서비스 'Lexi Note'이며, 우수상에는 벌레 퇴치 매칭 서비스 '우리집 히어로즈', 수업 소통 서비스 '에코 클래스룸'이 포함됩니다. 입선작으로는 동아리 정보 플랫폼 'Crewing', 학교 생활 관리 앱 '학교생활 매니저', 외국인과 한국인을 연결하는 플랫폼 'BLOTIE'가 있습니다.


# [MY CODE] RetrievalQA 사용

In [124]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,                # 언어 모델
    chain_type="stuff",     # 검색된 모든 문서를 합쳐 전달 ("stuff" 방식)
    retriever=retriever,    # 벡터 스토어 리트리버
    return_source_documents=True  # 답변에 사용된 문서 출처 반환
)

In [128]:
# OpenAI API 호출과 토큰 추적
with get_openai_callback() as callback:
    response = qa_chain.invoke({"query": user_msg})

    # 토큰 관련 정보 출력
    print(f"입력 토큰 수: {callback.prompt_tokens}")
    print(f"응답 토큰 수: {callback.completion_tokens}")
    print(f"총 토큰 수: {callback.total_tokens}")
    print(f"총 비용: ${callback.total_cost}")

입력 토큰 수: 10437
응답 토큰 수: 425
총 토큰 수: 10862
총 비용: $0.00133095


In [129]:
print("\n### 출처 문서 ###")
for doc in response["source_documents"]:
    print(f"출처: {doc.metadata.get('source', 'N/A')}")
    print(f"내용: {doc.page_content[:200]}")  # 문서 내용 일부 출력


### 출처 문서 ###
출처: https://spartacodingclub.kr/blog/all-in-challenge_winner
내용: 스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  493·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도
출처: https://spartacodingclub.kr/blog/all-in-challenge_winner
내용: 스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  493·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도
출처: https://spartacodingclub.kr/blog/all-in-challenge_winner
내용: 스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  493·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도
출처: https://spartacodingclub.kr/blog/all-in-challenge_winner
내용: 스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  493·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습

In [130]:
print(response["result"])

ALL-in 코딩 공모전에서는 다음과 같은 수상작들이 선정되었습니다:

1. **대상 - Lexi Note**: 언어 공부를 효율적으로 도와주는 필기 웹 서비스. 단어를 드래그하면 네이버 사전과 연결되고, 번역 기능도 제공하여 학습 속도를 향상시킬 수 있습니다. 기술 스택: React, Java, Spring Boot 등.

2. **우수상 - 우리집 히어로즈**: 벌레 문제를 해결하기 위한 영웅 매칭 서비스. 사용자와 같은 학교 학생들끼리 벌레 퇴치 요청을 하고 매칭되어 신원이 보장된 환경에서 문제를 해결할 수 있습니다. 기술 스택: React Native, Spring Boot 등.

3. **우수상 - 에코 클래스룸**: 교수와 학생 간의 실시간 소통 서비스를 제공, 익명으로 질문을 제출할 수 있도록 하여 학생들의 이해도를 체크할 수 있는 기능을 제공합니다. 기술 스택: Flutter, Spring Boot 등.

4. **입선 - Crewing**: 연합 동아리 정보를 제공하고 적절한 동아리를 쉽게 찾을 수 있도록 돕는 플랫폼. 사용자 맞춤형 추천과 동아리 후기를 통해 정보를 제공합니다. 기술 스택: Spring Boot, SwiftUI 등.

5. **입선 - 학교생활 매니저**: 학교 생활을 관리하는 앱으로, 일정, 과제, 성적 예측 등을 한 곳에서 관리할 수 있습니다. 기술 스택: Flutter, Firebase 등.

6. **입선 - BLOTIE**: 외국인과 내국인 학생 간의 매칭 및 교류를 촉진하는 플랫폼. 서로의 문화와 언어를 배우고 소통할 수 있는 기능을 제공합니다. 기술 스택: React Native, Spring Boot 등.

이 공모전은 대학생들이 캠퍼스에서 경험한 문제를 해결하고, 혁신적인 아이디어를 실현하는 데 중점을 두었습니다.
